In [1]:
import pandas as pd

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('Dane/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('Dane/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('Dane/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('Dane/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('Dane/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [2]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
items.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
Nk = users.shape[0]
Nk

943

In [6]:
Ni = items.shape[0]
Ni

1682

In [51]:
k = 30
learning_rate = 0.001

In [52]:
import numpy as np

In [58]:
U = (np.random.random((Nk, k)) - 0.5) / 10
#U

array([[ 0.00462419, -0.03665728,  0.03055792, ...,  0.0446104 ,
         0.0088018 ,  0.02796916],
       [ 0.03121887,  0.03537908,  0.0028825 , ..., -0.03743405,
         0.01596337,  0.03353265],
       [ 0.02976069, -0.0444041 , -0.03423131, ...,  0.04373534,
         0.04942211, -0.00489903],
       ..., 
       [-0.01247854,  0.01969643, -0.04877076, ..., -0.04053938,
        -0.0449584 , -0.01011615],
       [ 0.02607875,  0.02167635, -0.03370418, ...,  0.03942942,
         0.00541688, -0.00122751],
       [ 0.03511322,  0.04935026, -0.00738135, ...,  0.02605328,
        -0.04365953, -0.01835767]])

In [59]:
I = (np.random.random((k, Ni)) - 0.5) / 10
#I

array([[ 0.02641793,  0.02310492,  0.03849393, ..., -0.01607282,
         0.02408088, -0.04469139],
       [-0.04087882,  0.02960056,  0.02029566, ..., -0.02612535,
         0.00460035,  0.04684475],
       [-0.0059112 ,  0.03118646,  0.01429866, ..., -0.04830091,
        -0.04896707, -0.04251883],
       ..., 
       [-0.04430381, -0.04692093,  0.03734856, ...,  0.02662277,
        -0.03662566,  0.04405735],
       [-0.03437676,  0.03627527, -0.04552386, ...,  0.02292595,
         0.02439007,  0.02366973],
       [-0.0066346 ,  0.02164738, -0.0297455 , ..., -0.0157541 ,
         0.04837783,  0.00724254]])

In [60]:
U.shape, I.shape

((943, 30), (30, 1682))

In [61]:
for i in range(10):
    for index, row in ratings.iterrows():
        
        user_id = row['user_id'] - 1
        movie_id = row['movie_id'] - 1
        Rij = row['rating']

        us = U[user_id, :]
        it = I[:, movie_id]

        dCdu = 2 * (Rij - np.dot(us,it)) * -it
        dCdi = 2 * (Rij - np.dot(us,it)) * -us

        U[user_id, :] = U[user_id, :] - learning_rate * dCdu
        I[:,movie_id] = I[:,movie_id] - learning_rate * dCdi
    
    mse = 0
    Rhat = np.dot(U, I)
    for index, row in ratings.iterrows():
        user_id = row['user_id'] - 1
        movie_id = row['movie_id'] - 1
        Rij = row['rating']
        mse += (Rhat[user_id, movie_id] - Rij) ** 2
    mse /= ratings.shape[0]
    print(mse)

13.7312587246
13.7613357138
14.1440500294
24.8309381564


/Users/lukasz/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:12: RuntimeWarning: overflow encountered in multiply
/Users/lukasz/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: overflow encountered in multiply


nan
nan
nan
nan
nan
nan


In [ ]:
#Rozwiazanie Kuby:
# dla każdego wiersza w ratings:
# us - wiersz macierzy U o indeksie user id
# it - kolumna macierzy I o indeksie movie id
# dCdu - pochodna funkcji celu po us
# dCdi - pochodna funkcji celu po it
# U[user_id] = U[user_id] - learning_rate * dCdu
# I[:, movie_id] = I[:, movie_id] - learning_rate * dCdi

k = 10 # liczba cech ukrytych
U = (np.random.sample((users_no, k)) -.5) / 10
I = (np.random.sample((k, movies_no)) -.5) / 10

learning_rate = 0.001

for epoch in range(10):
    for row in range(ratings.shape[0]):
        us = ratings.iloc[row]['user_id']
        it = ratings.iloc[row]['movie_id']
        dCdu = 2 * (ratings.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1])) * (-I[:, it-1])
        dCdi = 2 * (ratings.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1])) * (-U[us-1,:])
        U[us-1,:] = U[us-1,:] - learning_rate * dCdu
        I[:, it-1] = I[:, it-1] - learning_rate * dCdi
    min_ = 0
    for row in range(ratings.shape[0]):
        us = ratings.iloc[row]['user_id']
        it = ratings.iloc[row]['movie_id']
        min_ += (ratings.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1]))**2
    min_ /= ratings.shape[0]
    print("Epoch:", epoch + 1, "Cost: ", min_ )

In [66]:
users_no = users.shape[0]
movies_no = items.shape[0]
k = 30 # liczba cech ukrytych
U = (np.random.sample((users_no, k)) -.5) / 10
I = (np.random.sample((k, movies_no)) -.5) / 10

learning_rate = 0.001

for epoch in range(55):
    for row in range(ratings_train.shape[0]):
        us = ratings_train.iloc[row]['user_id']
        it = ratings_train.iloc[row]['movie_id']
        dCdu = 2 * (ratings_train.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1])) * (-I[:, it-1])
        dCdi = 2 * (ratings_train.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1])) * (-U[us-1,:])
        U[us-1,:] = U[us-1,:] - learning_rate * dCdu
        I[:, it-1] = I[:, it-1] - learning_rate * dCdi
        
    min_test = 0
    for row in range(ratings_test.shape[0]):
        us = ratings_test.iloc[row]['user_id']
        it = ratings_test.iloc[row]['movie_id']
        min_test += (ratings_test.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1]))**2
    min_test /= ratings_test.shape[0]
    
    min_train = 0
    for row in range(ratings_train.shape[0]):
        us = ratings_train.iloc[row]['user_id']
        it = ratings_train.iloc[row]['movie_id']
        min_train += (ratings_train.iloc[row]['rating'] - np.dot(U[us-1,:], I[:,it-1]))**2
    min_train /= ratings_train.shape[0]
    print("Epoch:", epoch + 1, "Train Cost: ", min_train, "Test Cost: ", min_test )
    # uwazac na przeuczenie na danych treningowych

Epoch: 1 Cost:  14.1265151579
Epoch: 2 Cost:  14.1135686671
Epoch: 3 Cost:  13.9702537731
Epoch: 4 Cost:  12.7341597556
Epoch: 5 Cost:  8.94005182149
Epoch: 6 Cost:  6.14544567879
Epoch: 7 Cost:  4.63781723665
Epoch: 8 Cost:  3.69445319629
Epoch: 9 Cost:  3.05863066541
Epoch: 10 Cost:  2.60929841366
Epoch: 11 Cost:  2.28021991919
Epoch: 12 Cost:  2.03225253305
Epoch: 13 Cost:  1.84103693837
Epoch: 14 Cost:  1.69077059347
Epoch: 15 Cost:  1.57082662507
Epoch: 16 Cost:  1.47382911542
Epoch: 17 Cost:  1.3945147445
Epoch: 18 Cost:  1.32903609838
Epoch: 19 Cost:  1.27452162404
Epoch: 20 Cost:  1.22878942904
Epoch: 21 Cost:  1.19015602987
Epoch: 22 Cost:  1.15730534352
Epoch: 23 Cost:  1.12919691877
Epoch: 24 Cost:  1.10500036296
Epoch: 25 Cost:  1.08404765362
Epoch: 26 Cost:  1.06579790364
Epoch: 27 Cost:  1.049810939
Epoch: 28 Cost:  1.03572718799
Epoch: 29 Cost:  1.02325212668
Epoch: 30 Cost:  1.01214403082
Epoch: 31 Cost:  1.00220414361
Epoch: 32 Cost:  0.99326863566
Epoch: 33 Cost:  0.9